<a href="https://colab.research.google.com/github/Majeed2096/Artathon-Style-Transfer-/blob/master/tweet%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import nltk
import numpy as np


In [2]:
df = pd.read_csv('/content/tweet-featues.csv')
df1 = pd.read_csv('/content/topic-features-news.csv')

In [3]:

df.drop("Topic_Detection" , inplace=True , axis = 1)
df.drop("tweet_id" , inplace=True , axis = 1)
df.drop("tweet" , inplace=True , axis = 1)
df['user_Nametype'] = df['user_Nametype'].astype('category')
df['tweet_type'] = df['tweet_type'].astype('category')



df.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107150 entries, 0 to 107149
Data columns (total 32 columns):
 #   Column                           Non-Null Count   Dtype   
---  ------                           --------------   -----   
 0   Topic_Name                       107150 non-null  object  
 1   user_Nametype                    107150 non-null  category
 2   tweet_without_EOL                107150 non-null  object  
 3   tweet_type                       107150 non-null  category
 4   tweet_length                     107150 non-null  int64   
 5   tweet_contains_mention           107150 non-null  int64   
 6   tweet_contains_URL               107150 non-null  int64   
 7   tweet_contains_hashtag           107150 non-null  int64   
 8   tweet_is_retweet                 107150 non-null  int64   
 9   tweet_contains_multimedia        107150 non-null  int64   
 10  Tweet_contains_NSFW_content      107150 non-null  int64   
 11  tweet_contains_question_marks    107150 non-null  in

In [4]:
df.head(2)

,Topic_Name,user_Nametype,tweet_without_EOL,tweet_type,tweet_length,tweet_contains_mention,tweet_contains_URL,tweet_contains_hashtag,tweet_is_retweet,tweet_contains_multimedia,Tweet_contains_NSFW_content,tweet_contains_question_marks,tweet_sentiment,tweet_contains_exclamation,tweet_contains_happy_smileys,tweet_contains_sad_smileys,tweet_has_specific_words,Authors_hasURL_in_his_home_page,The_number_of_favorites,Is_Author_Credible,registration_age,Number_of_followers,Number_of_following,Number_of_retweets,Reputaion_Score,Is_authors_has_description,Author_has_Notification,Author_followed_by_credibleuser,Time_span,listed_count,Number_of_tweet_posted,User_effect
0,شاب سعودي ينقذ طفل من امام القطار في بلجيكا,Personal,شجاعة شاب سعودي ينقذ طفلاً في بلجيكا قبل أن يد...,News,209,0,1,1,0,0,0,0,Positive,0,0,0,0,0,1,0,1,6877,296,3,0.96,1,0,0,1,4,3210,23.23
1,شاب سعودي ينقذ طفل من امام القطار في بلجيكا,Personal,شجاعة شاب سعودي ينقذ طفلاً في بلجيكا قبل أن ي...,News,233,0,0,1,1,0,0,0,Positive,0,0,0,0,0,0,0,5,1108,2375,3,0.32,1,0,0,5,2,16212,0.47


In [5]:
df = df.merge(df1 , on = "Topic_Name")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107150 entries, 0 to 107149
Data columns (total 62 columns):
 #   Column                                                    Non-Null Count   Dtype   
---  ------                                                    --------------   -----   
 0   Topic_Name                                                107150 non-null  object  
 1   user_Nametype                                             107150 non-null  category
 2   tweet_without_EOL                                         107150 non-null  object  
 3   tweet_type                                                107150 non-null  category
 4   tweet_length                                              107150 non-null  int64   
 5   tweet_contains_mention                                    107150 non-null  int64   
 6   tweet_contains_URL                                        107150 non-null  int64   
 7   tweet_contains_hashtag                                    107150 non-null  int64   

In [6]:
df.groupby("Topic_Detection").count()
df.head(1)

,Topic_Name,user_Nametype,tweet_without_EOL,tweet_type,tweet_length,tweet_contains_mention,tweet_contains_URL,tweet_contains_hashtag,tweet_is_retweet,tweet_contains_multimedia,Tweet_contains_NSFW_content,tweet_contains_question_marks,tweet_sentiment,tweet_contains_exclamation,tweet_contains_happy_smileys,tweet_contains_sad_smileys,tweet_has_specific_words,Authors_hasURL_in_his_home_page,The_number_of_favorites,Is_Author_Credible,registration_age,Number_of_followers,Number_of_following,Number_of_retweets,Reputaion_Score,Is_authors_has_description,Author_has_Notification,Author_followed_by_credibleuser,Time_span,listed_count,Number_of_tweet_posted,User_effect,Ratio_orgnization_users_Nametype,Ratio_of_tweets_containing_mention,Ratio_of_tweets_containing_URLs,Ratio_of_tweets_containing_hashtag,Ratio_of_tweet_containing_NSFW_content,Ratio_of_tweets_that_are_retweets,Ratio_of_tweets_have_multimedia,Ratio_of_tweets_from_credible_authors,Ratio_of_tweets_containing_question_marks,Ratio_of_tweets_with_positive_sentimnet,Ratio_of_tweets_with_negative_sentiment,Ratio_of_tweets_with_netural_sentiment,Ratio_of_tweets_containing_exclamation,Ratio_of_tweets_containing_happy_smileys,Ratio_of_tweets_having_specific_words_in_tweet,Ratio_of_tweets_from_authors_have_URL_in_their_home_page,Ratio_of_Authorsr_who_follwed_by__credibleuser,Ratio_of_authors_have_description,Average_tweet_length,Average_registration_age,Average_number_of_tweets_posted_Activity,Average_number_of_favorites,Average_Number_of_followers,Average_Number_of_following,Average_Time_span,Average_number_of_retweets,Average_listed_counts,Average_user_effect,Average_of_Reputaion,Topic_Detection
0,شاب سعودي ينقذ طفل من امام القطار في بلجيكا,Personal,شجاعة شاب سعودي ينقذ طفلاً في بلجيكا قبل أن يد...,News,209,0,1,1,0,0,0,0,Positive,0,0,0,0,0,1,0,1,6877,296,3,0.96,1,0,0,1,4,3210,23.23,0.008,0.033,0.114,0.553,0.024,0.789,0.268,0.0,0.041,0.943,0.041,0.016,0.008,0.0,0.0,0.073,0.0,1.0,205.041,2.366,18789.1,1.943,4635.43,1442.63,2.366,17.366,4.683,1831.28,0.492,Rumor


In [7]:
for column in df:
  if (( str(df[column].dtypes) == 'int64') | (str(df[column].dtypes) == 'float64')):
    df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    df[column]  = df[column].fillna(0)

df.head(1)

,Topic_Name,user_Nametype,tweet_without_EOL,tweet_type,tweet_length,tweet_contains_mention,tweet_contains_URL,tweet_contains_hashtag,tweet_is_retweet,tweet_contains_multimedia,Tweet_contains_NSFW_content,tweet_contains_question_marks,tweet_sentiment,tweet_contains_exclamation,tweet_contains_happy_smileys,tweet_contains_sad_smileys,tweet_has_specific_words,Authors_hasURL_in_his_home_page,The_number_of_favorites,Is_Author_Credible,registration_age,Number_of_followers,Number_of_following,Number_of_retweets,Reputaion_Score,Is_authors_has_description,Author_has_Notification,Author_followed_by_credibleuser,Time_span,listed_count,Number_of_tweet_posted,User_effect,Ratio_orgnization_users_Nametype,Ratio_of_tweets_containing_mention,Ratio_of_tweets_containing_URLs,Ratio_of_tweets_containing_hashtag,Ratio_of_tweet_containing_NSFW_content,Ratio_of_tweets_that_are_retweets,Ratio_of_tweets_have_multimedia,Ratio_of_tweets_from_credible_authors,Ratio_of_tweets_containing_question_marks,Ratio_of_tweets_with_positive_sentimnet,Ratio_of_tweets_with_negative_sentiment,Ratio_of_tweets_with_netural_sentiment,Ratio_of_tweets_containing_exclamation,Ratio_of_tweets_containing_happy_smileys,Ratio_of_tweets_having_specific_words_in_tweet,Ratio_of_tweets_from_authors_have_URL_in_their_home_page,Ratio_of_Authorsr_who_follwed_by__credibleuser,Ratio_of_authors_have_description,Average_tweet_length,Average_registration_age,Average_number_of_tweets_posted_Activity,Average_number_of_favorites,Average_Number_of_followers,Average_Number_of_following,Average_Time_span,Average_number_of_retweets,Average_listed_counts,Average_user_effect,Average_of_Reputaion,Topic_Detection
0,شاب سعودي ينقذ طفل من امام القطار في بلجيكا,Personal,شجاعة شاب سعودي ينقذ طفلاً في بلجيكا قبل أن يد...,News,0.637324,0.0,1.0,1.0,0.0,0.0,0.0,0.0,Positive,0.0,0.0,0.0,0.0,0.0,0.000384,0.0,0.090909,0.000383,0.000133,0.000322,0.96,1.0,0.0,0.0,0.090909,0.000216,0.001465,0.000002,0.008,0.033,0.114,0.553,0.167832,0.789,0.268,0.0,0.041,0.943,0.041,0.016,0.008889,0.0,0.0,0.073,0.0,1.0,0.727681,0.275116,0.048511,0.140889,0.006747,0.040924,0.275116,0.004387,0.006817,0.021863,0.454248,Rumor


In [8]:
df['tweet_sentiment'] = df.tweet_sentiment=='Positive'
df['Topic_Detection'] = df.Topic_Detection=='Rumor'
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107150 entries, 0 to 107149
Data columns (total 62 columns):
 #   Column                                                    Non-Null Count   Dtype   
---  ------                                                    --------------   -----   
 0   Topic_Name                                                107150 non-null  object  
 1   user_Nametype                                             107150 non-null  category
 2   tweet_without_EOL                                         107150 non-null  object  
 3   tweet_type                                                107150 non-null  category
 4   tweet_length                                              107150 non-null  float64 
 5   tweet_contains_mention                                    107150 non-null  float64 
 6   tweet_contains_URL                                        107150 non-null  float64 
 7   tweet_contains_hashtag                                    107150 non-null  float64 

In [9]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
str(df['Topic_Name'].dtypes)

'object'

In [12]:
stemed_text_topic = []
for i in df['Topic_Name']:
  new_stemed_sentense = []
  for j in i.split(' '):
    if not j in stopwords.words('arabic'): 
      new_stemed_sentense.append(PorterStemmer().stem(j))
  stemed_text_topic.append(" ".join(new_stemed_sentense))   

In [13]:
stemed_text_tweet = []
for i in df['tweet_without_EOL']:
  new_stemed_sentense = []
  for j in i.split(' '):
    if not j in stopwords.words('arabic'): 
      new_stemed_sentense.append(PorterStemmer().stem(j))
  stemed_text_tweet.append(" ".join(new_stemed_sentense))  

# New Section

In [14]:
df['Topic_Name'] = stemed_text_topic
df['tweet_without_EOL'] = stemed_text_tweet

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000)

In [17]:
hashtag = cv.fit_transform(df['Topic_Name'].values).toarray()
cv = CountVectorizer(max_features=1000)
tweet = cv.fit_transform(df['tweet_without_EOL'].values).toarray()

In [18]:
tweet.shape

(107150, 2500)

In [19]:
x = np.concatenate((hashtag,tweet),axis=1)

In [20]:
feture = pd.DataFrame()
feture = df.drop("Topic_Name" , axis = 1)
feture = feture.drop("tweet_without_EOL" , axis = 1)
feture = feture.drop("Topic_Detection" , axis = 1)
feture = pd.concat([feture, pd.DataFrame(x)], axis=1)
feture.shape

(107150, 3426)

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(feture, df['Topic_Detection'], test_size=0.33, random_state=42)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [23]:
random_forest = RandomForestClassifier()

In [ ]:
random_forest.fit(x_train,y_train)

In [ ]:
pred = random_forest.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,pred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test,pred, average='weighted')
